In [1]:
import numpy as np
import pandas as pd
import dill

from sklearn.metrics import precision_recall_curve

In [2]:
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

In [3]:
x_test.head()

,Age,Gender,Family History,Personal History,Current Stressors,Symptoms,Severity,Impact on Life,Demographics,Medical History,Psychiatric History,Substance Use,Coping Mechanisms,Social Support,Lifestyle Factors
0,21,Female,No,Yes,Moderate,Dizziness,Severe,Significant,Urban,Heart disease,NaN,Alcohol,Socializing,High,Diet
1,31,Male,Yes,Yes,Low,Fear of losing control,Mild,Significant,Rural,Asthma,Bipolar disorder,Alcohol,Meditation,High,Sleep quality
2,19,Male,No,Yes,Low,Dizziness,Mild,Significant,Rural,Diabetes,Depressive disorder,Drugs,Socializing,Low,Exercise
3,25,Male,No,Yes,Moderate,Panic attacks,Severe,Significant,Urban,Asthma,Depressive disorder,NaN,Socializing,Low,Diet
4,21,Male,No,Yes,Low,Fear of losing control,Severe,Moderate,Urban,NaN,Bipolar disorder,NaN,Socializing,High,Exercise


In [4]:
with open('svc_clf.dill', 'rb') as model:
    pipeline = dill.load(model)

In [5]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Family History',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Family '
                                                                                         'History')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Family '
                                                                                 'History'))])),
                                                ('Personal History',
                                                 Pipeline(steps=[('selector',
                                                                  Fea...
                                                                  FeatureSelector(column='Social '
                                                                                         'Support')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Social '
                                                                                 'Support'))])),
                                                ('Lifestyle Factors',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Lifestyle '
                                                                                         'Factors')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Lifestyle '
                                                                                 'Factors'))])),
                                                ('Age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Age')),
                                                                 ('Scale',
                                                                  StandardScaler())]))])),
                ('classifier', SVC(probability=True))])

In [6]:
preds = pipeline.predict_proba(x_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv('test_predictions.csv', index=None)
preds[:10]

array([1.52381490e-05, 9.55716710e-03, 3.33294223e-05, 1.44114930e-03,
       4.96904149e-07, 1.00000010e-07, 1.66473376e-07, 8.68358757e-06,
       2.77383103e-03, 4.54033388e-07])

In [7]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)

ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]:.3f}, F-Score={fscore[ix]:.3f}, '
      f'Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.402, F-Score=0.939, Precision=0.889, Recall=0.995
